In [ ]:
import tensorflow as tf
import os
import numpy as np
import matplotlib.pyplot as plt
import tifffile as tiff
import pandas as pd
import json



In [ ]:
#dataset link
url= "https://zenodo.org/records/7711810/files/EuroSAT_MS.zip?download=1"

In [ ]:
tf.config.list_physical_devices()

In [ ]:
cd /workspace/solution

In [ ]:
ls

In [ ]:
#load the images using tf.data.Dataset
data_dir = "EuroSAT_MS"

#list the files in the directory
file_list = tf.io.gfile.glob(data_dir + '/*/*.tif')

In [ ]:
#writie a metadata file
import pandas as pd
df = pd.DataFrame(file_list, columns=["file_path"])

In [ ]:
#split the file path to get the class name
df["class_name"] = df["file_path"].apply(lambda x: x.split("/")[-2])

In [ ]:
#encode the class names
class_names = df["class_name"].unique()
class_names
#initialize the label encoder
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
label_encoder.fit(class_names)
label_encoder.classes_
df["encoded"]=label_encoder.fit_transform(df["class_name"])


In [ ]:
#save the encoder to a json and text to csv
import json
with open("label_encoder.json", "w") as f:
    json.dump(label_encoder.classes_.tolist(), f)
df.to_csv("metadata.csv", index=False)



In [ ]:
df

In [ ]:
df=pd.read_csv("metadata.csv")
encoder = json.load(open("label_encoder.json", "r"))

In [ ]:
dset={"img":[],
      "label":[]}
for i in range(len(df)):
    img = df["file_path"][i]
    label = df["encoded"][i]
    dset["img"].append(tiff.imread(img))
    dset["label"].append(label)
dset["img"]=np.array(dset["img"])

In [ ]:
#visualize the image with all the bands

bands =["ultra_blue", "blue", "green", "red", "705", "740", "783", "842", "865", "940", "1375", "1610", "2190","fertility"]
for i in range(len(dset["img"][1000:10002])):
    img =dset["img"][i]
    fig, axes = plt.subplots(1, len(bands), figsize=(12, 2), dpi=300)
    fig.suptitle("Image with all bands "+encoder[(dset["label"][0])])
    nvdi=(img[:,:,7]-img[:,:,3])/(img[:,:,3]+img[:,:,7])
    for i in range(len(bands)-1):
        axes[i].imshow(img[:,:,i])
        axes[i].set_title(bands[i])
        axes[i].axis("off")
    axes[i+1].imshow(nvdi)
    axes[i+1].set_title(bands[i+1])
    axes[i+1].axis("off")
    plt.show()
"""fig, axes = plt.subplots(1, len(bands)-1, figsize=(10, 1), dpi=300)
for i in range(len(bands)-1):
    axes[i].imshow(img[:,:,i])
    axes[i].set_title(bands[i])
    axes[i].axis("off")
"""

In [ ]:
dset["img"]=np.array(dset["img"])
dset["label"]=np.array(dset["label"])
dset["img"]=dset["img"].astype(np.uint16)
dset["label"]=dset["label"].astype(np.uint8)

In [ ]:
stratergy = tf.distribute.MirroredStrategy()
print('Number of devices: {}'.format(stratergy.num_replicas_in_sync))


In [ ]:

def create_model():
    # Load pre-trained MobileNetV2
    base_model = tf.keras.applications.EfficientNetV2S(input_shape=[64,64,13], include_top=False,weights=None)

    # Add layers to predict bounding boxes and class probabilities
    x = base_model.output
    x = tf.keras.layers.Flatten()(x)
    x= tf.keras.layers.Dense(10,"sigmoid")(x)

    # Create model
    model = tf.keras.Model(inputs=base_model.input, outputs=x)
    return model
model = create_model()
model.compile(optimizer='adam',loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),metrics=['accuracy'])


In [ ]:
len(encoder)

In [ ]:

a=model.fit(dset["img"], dset["label"], epochs=15,batch_size=512,shuffle=True,validation_split=0.1,ca)


In [ ]:
model.save("net.keras")